<a href="https://colab.research.google.com/github/Abonvicine/ESTUDOS---Gerais/blob/main/Teste_extraindo_dados_do_MERGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando CONDA e IRIS (opcional)

In [ ]:
!python --version # now returns Python 3.6.10 :: Anaconda, Inc.

Python 3.7.10


In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

Process is interrupted.


In [ ]:
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes

bash: line 1: conda: command not found
bash: line 2: conda: command not found


In [ ]:
import sys
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [ ]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.6/site-packages"))

# Dependencias iniciais

In [ ]:
!apt-get install -q libgeos-3.5.0
!apt-get install -q libgeos-dev
!pip install -q https://github.com/matplotlib/basemap/archive/master.zip
!pip install -q pyproj==1.9.6

In [ ]:
!pip install pygrib
!pip install geojsoncontour
!pip install cmocean

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import urllib.request as req
from datetime import datetime
import geojsoncontour
import cmocean
import pygrib
import shapely as shp
import numpy as np
import pandas as pd
import os

In [ ]:
#@title Inserir data do arquivo

DataStr = '2021-04-29' #@param {type:"date"}
Data = datetime.strptime(DataStr, '%Y-%m-%d')

In [ ]:
dia = str(Data.day).zfill(2)
mes = str(Data.month).zfill(2)
ano = str(Data.year)

if not(os.path.isdir("/content/MERGE")):
    os.mkdir("/content/MERGE")

arquivos = [
            f"MERGE_CPTEC_{ano}{mes}{dia}.ctl",
            f"MERGE_CPTEC_{ano}{mes}{dia}.grib2",
            f"MERGE_CPTEC_{ano}{mes}{dia}.idx"
            ]

for i in arquivos:
    url = f'http://ftp.cptec.inpe.br/modelos/tempo/MERGE/GPM/DAILY/2021/{mes}/{i}'
    req.urlretrieve(url, f'/content/MERGE/{i}')

gr = pygrib.open(f"/content/MERGE/{arquivos[1]}")  

In [ ]:
gr = pygrib.open("/content/MERGE_CPTEC_2021042608.grib2")  

# Extraindo Dados do Arquivo .GRIB2

## Arquivo GRIB2

In [ ]:
grb = gr.select(name='Precipitation')[0]
lat1 = -40
lat2 =  20
lon1 = 280
lon2 = 330
data, lats, lons = grb.data(lat1=lat1,lat2=lat2,lon1=lon1,lon2=lon2)
print(grb)

1:Precipitation:kg m**-2 (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 202104291200


In [ ]:
##testando pontos obtidos
print(f"({lats[0,0]},{lons[0,0]}) -> precipitação:{data[0,0]}")

(-39.94999999999971,280.05000000000456) -> precipitação:4.8125


In [ ]:
dataList = []

flatten_lons = lons.flatten()
flatten_lats = lats.flatten()
flatten_data = data.flatten()

for i in range(flatten_data.shape[0]):

    lat = round(flatten_lats[i],2)
    lon = round(flatten_lons[i],2)

    lon = round(((lon + 180) % 360) - 180,2) #transforma longitude de 0/360 para -180/180

    precipitation = round(flatten_data[i],2)

    dataList.append([lat,lon,precipitation])

In [ ]:
from google.colab import files

fileName = f'{dia}{mes}{ano}_MERGE.txt'
filePath = f"/content/drive/MyDrive/CPFL/ARQUIVOS ONS/PRECIPITAÇÃO/Modelo MERGE/{fileName}"
pd.DataFrame.from_dict(dataList).to_csv(filePath, index=False)
files.download(filePath)  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
MERGE_path = "/content/drive/MyDrive/CPFL/ARQUIVOS ONS/PRECIPITAÇÃO/Modelo MERGE/MERGE_interpolado.csv"

In [ ]:
model = pd.read_csv(MERGE_path, encoding="UTF-8")
model.drop(columns=["Unnamed: 0"])

,LAT,LNG,ARQUIVO,BACIA
0,-28.25,-52.25,Ita.bln,Uruguai
1,-28.25,-52.15,Ita.bln,Uruguai
2,-28.25,-52.05,Ita.bln,Uruguai
3,-28.25,-51.95,Ita.bln,Uruguai
4,-28.25,-51.85,Ita.bln,Uruguai
...,...,...,...,...
30840,-20.95,-43.85,Funil.bln,Grande
30841,-20.95,-43.75,Funil.bln,Grande
30842,-20.85,-44.75,Funil.bln,Grande
30843,-20.85,-44.45,Funil.bln,Grande


In [ ]:
observed = model.merge(df)
fileName = f'{dia}{mes}{ano}_MERGE.dat'
observed.to_csv(fileName, index=False)
files.download(fileName)  

## Plotando MAPA

In [ ]:
# Função Basemap, plotar a projeção, o recorte das latitude e longitude e definir a resolução da imagem.
m = Basemap(projection='mill',llcrnrlat=lat1,urcrnrlat=lat2,\
            llcrnrlon=lon1,urcrnrlon=lon2,resolution='i')

# Função Matplotlib para definir o tamanho da figura.
plt.figure(figsize=(20,30))

#Inserindo os lat e lon a figura criada pelo Basemap e vinculando às variavéis  x e y.
x, y = m(lons, lats)
#Função do Basemap para iserir ao mapa continentes, países e estados.
m.drawcoastlines()
m.drawcountries(linewidth=2.0)
m.drawstates(linewidth=1.0)
m.drawrivers(color = "#0009b3")

#Função do Basemap para realizar a interpolação e criação de polígonos
levels = np.linspace(0, 200, 51)
#colorSchema = cmocean.cm.dense
contourf = m.contourf(x, y, np.squeeze(data), levels = [0,1,5,10,20,50,100,200], colors=["#ffffff","#d5eb34",'#fccf03',"#baa60b", '#03fc2c', '#0380fc',"#ba0b0b"])
cbar = m.colorbar(location='right', pad="10%")

for t in cbar.ax.get_yticklabels():
     t.set_fontsize(20)
     
#Inserir titulo a imagem 
plt.title(f"MERGE (precipitação Observada) — {dia}-{mes}-{ano} - Acumulado 24h",fontsize= 30)
#Convertendo para o formato Geojson
geojson = geojsoncontour.contourf_to_geojson(
    contourf=contourf,
    min_angle_deg=3.0,
    ndigits=3,
    stroke_width=3,
)

#Plotar imagem
plt.show()

## Interpolando o Arquivo 

In [ ]:
def mergeCoord(lat,lon):
    return f"{str(lat)}:{str(lon)}"

In [ ]:
previsao = pd.DataFrame(dataList,columns=['LAT','LNG',"PRECIPITAÇÃO"])
previsao["COORD"] = previsao.apply(lambda x: mergeCoord(x.LAT,x.LNG), axis =1 )
previsao = previsao[["COORD","PRECIPITAÇÃO"]]

fileName = f'/content/drive/MyDrive/CPFL/ARQUIVOS ONS/PRECIPITAÇÃO/Contornos/25042021_MERGE_interpolado.csv'
pontos_de_grade = pd.read_csv(fileName)
pontos_de_grade["COORD"] = pontos_de_grade.apply(lambda x: mergeCoord(x.LAT,x.LNG), axis =1 )
pontos_de_grade = pontos_de_grade[["COORD","ARQUIVO","BACIA"]]

df = pd.merge(pontos_de_grade, previsao, on='COORD', how="inner")

In [ ]:
print(previsao.shape)
print(pontos_de_grade.shape)
print(df.shape)

(300000, 2)
(30846, 3)
(30846, 4)


In [ ]:
filtro = df[df["BACIA"] =='Tocantins']
print(filtro.mean()["PRECIPITAÇÃO"])
filtro

In [ ]:
reservatorios = df["ARQUIVO"].unique()

pMEDIA_res = {"RESERVATORIO":[],"PRECIPITAÇÃO":[]}
for res in reservatorios:
    filtro = df[df["ARQUIVO"] == res]
    media = filtro.mean()["PRECIPITAÇÃO"]
    pMEDIA_res["RESERVATORIO"].append(res)
    pMEDIA_res["PRECIPITAÇÃO"].append(media)

In [ ]:
bacias = df["BACIA"].unique()

pMEDIA_bac = {"RESERVATORIO":[],"PRECIPITAÇÃO":[]}
for bac in bacias:
    filtro = df[df["BACIA"] == bac]
    media = filtro.mean()["PRECIPITAÇÃO"]
    pMEDIA_bac["RESERVATORIO"].append(bac)
    pMEDIA_bac["PRECIPITAÇÃO"].append(media)

In [ ]:
pMEDIA = pd.DataFrame.from_dict(pMEDIA_bac)
pMEDIA.to_excel("MERGE_PMEDIA.xlsx")
pMEDIA.to_csv("MERGE_PMEDIA.csv")